In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import FastICA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import decomposition
from sklearn.feature_selection import SelectPercentile as sp
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFECV
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('Network_dataset_1.csv')

In [3]:
df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [4]:
df = df.drop(['ts', 'src_ip', 'src_port', 'dst_ip', 'dst_port'], axis = 1)
df

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,-,-,-,-,-,-,0,normal
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,tcp,-,0.000000,0,0,S0,0,1,48,0,...,0,0,-,-,-,-,-,-,1,scanning
999996,tcp,-,0.000000,0,0,S0,0,1,48,0,...,0,0,-,-,-,-,-,-,1,scanning
999997,tcp,-,0.000000,0,0,S0,0,1,48,0,...,0,0,-,-,-,-,-,-,1,scanning
999998,tcp,-,0.000000,0,0,S0,0,1,48,0,...,0,0,-,-,-,-,-,-,1,scanning


In [5]:
df = df.replace('-', np.NaN)

In [6]:
df.isnull().sum()

proto                          0
service                   935866
duration                       0
src_bytes                      0
dst_bytes                      0
conn_state                     0
missed_bytes                   0
src_pkts                       0
src_ip_bytes                   0
dst_pkts                       0
dst_ip_bytes                   0
dns_query                 959246
dns_qclass                     0
dns_qtype                      0
dns_rcode                      0
dns_AA                    959032
dns_RD                    959032
dns_RA                    959032
dns_rejected              959032
ssl_version               999601
ssl_cipher                999601
ssl_resumed               999083
ssl_established           999083
ssl_subject               999991
ssl_issuer                999991
http_trans_depth          999749
http_method               999765
http_uri                  999765
http_version              999749
http_request_body_len          0
http_respo

In [7]:
df = df.dropna(axis = 1)

In [8]:
df.isnull().sum()

proto                     0
duration                  0
src_bytes                 0
dst_bytes                 0
conn_state                0
missed_bytes              0
src_pkts                  0
src_ip_bytes              0
dst_pkts                  0
dst_ip_bytes              0
dns_qclass                0
dns_qtype                 0
dns_rcode                 0
http_request_body_len     0
http_response_body_len    0
http_status_code          0
label                     0
type                      0
dtype: int64

In [9]:
target = df.loc[:, ('type')]

In [10]:
df = df.drop('type', axis = 1)

In [11]:
cat = []
for i in df.columns:
    if len(df[i].unique()) <= 200: cat.append(i)
df_cat = df[cat]
df_cat

,proto,conn_state,dns_qclass,dns_qtype,dns_rcode,http_request_body_len,http_response_body_len,http_status_code,label
0,tcp,OTH,0,0,0,0,0,0,0
1,udp,S0,0,0,0,0,0,0,0
2,tcp,OTH,0,0,0,0,0,0,0
3,tcp,OTH,0,0,0,0,0,0,0
4,udp,SHR,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
999995,tcp,S0,0,0,0,0,0,0,1
999996,tcp,S0,0,0,0,0,0,0,1
999997,tcp,S0,0,0,0,0,0,0,1
999998,tcp,S0,0,0,0,0,0,0,1


In [12]:
df_temp = df.drop(df_cat, axis = 1)

In [13]:
scaler = StandardScaler()
df[df_temp.columns] = scaler.fit_transform(df[df_temp.columns])

In [14]:
df[df_temp.columns] = preprocessing.normalize(df[df_temp.columns])

In [15]:
df

,proto,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_qclass,dns_qtype,dns_rcode,http_request_body_len,http_response_body_len,http_status_code,label
0,tcp,0.491219,0.007327,0.145894,OTH,-0.000005,0.837575,0.189296,-0.000003,-0.000005,0,0,0,0,0,0,0
1,udp,-0.700390,-0.235421,-0.298467,S0,-0.229351,-0.278497,-0.257794,-0.326311,-0.248696,0,0,0,0,0,0,0
2,tcp,-0.661265,-0.222270,-0.281794,OTH,-0.216539,-0.400624,-0.276549,-0.308083,-0.234804,0,0,0,0,0,0,0
3,tcp,-0.661265,-0.222270,-0.281794,OTH,-0.216539,-0.400624,-0.276549,-0.308083,-0.234804,0,0,0,0,0,0,0
4,udp,-0.705378,-0.237147,-0.254797,SHR,-0.231033,-0.427440,-0.295059,-0.149652,-0.189062,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,tcp,-0.698861,-0.234907,-0.297815,S0,-0.228850,-0.277889,-0.265574,-0.325599,-0.248153,0,0,0,0,0,0,1
999996,tcp,-0.698861,-0.234907,-0.297815,S0,-0.228850,-0.277889,-0.265574,-0.325599,-0.248153,0,0,0,0,0,0,1
999997,tcp,-0.698861,-0.234907,-0.297815,S0,-0.228850,-0.277889,-0.265574,-0.325599,-0.248153,0,0,0,0,0,0,1
999998,tcp,-0.698861,-0.234907,-0.297815,S0,-0.228850,-0.277889,-0.265574,-0.325599,-0.248153,0,0,0,0,0,0,1


In [16]:
label_encoder = preprocessing.LabelEncoder()
df['proto'] = label_encoder.fit_transform(df['proto'])
df['conn_state'] = label_encoder.fit_transform(df['conn_state'])

In [17]:
def lr(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_lr = LogisticRegression()
    model_lr.fit(x_train, y_train)
    preds = model_lr.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [18]:
def dtc(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_dtc = DecisionTreeClassifier()   
    model_dtc.fit(x_train, y_train)
    preds = model_dtc.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [19]:
def rfc(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_rfc = RandomForestClassifier()    
    model_rfc.fit(x_train, y_train)
    preds = model_rfc.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [20]:
def knn(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_knn = KNeighborsClassifier()
    model_knn.fit(x_train, y_train)
    preds = model_knn.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [21]:
def svm(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_svm = SVC(kernel = 'sigmoid')
    model_svm.fit(x_train, y_train) 
    preds = model_svm.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [22]:
def nb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_nb = GaussianNB()
    model_nb.fit(x_train, y_train)
    preds = model_nb.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [23]:
def xgbb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_xgb = xgb.XGBClassifier()
    model_xgb.fit(x_train, y_train)
    preds = model_xgb.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [24]:
def adb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_adb = AdaBoostClassifier()
    model_adb.fit(x_train, y_train)
    preds = model_adb.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [25]:
def mlp(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_mlp = MLPClassifier()    
    model_mlp.fit(x_train, y_train)
    preds = model_mlp.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [26]:
def gb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_mlp = GradientBoostingClassifier() 
    model_mlp.fit(x_train, y_train)
    preds = model_mlp.predict(x_test)
    return accuracy_score(y_test, preds), f1_score(y_test, preds, average = 'macro')

In [27]:
target = label_encoder.fit_transform(target)

In [28]:
x = df
y = target

In [29]:
x.columns = x.columns.astype('str')

In [30]:
#without feature selection


print('lr - ', lr(x, y))
print('dtc - ', dtc(x, y))
print('rfc - ', rfc(x, y))
print('knn - ', knn(x, y))
# print('svm - ', svm(x, y))
print('Naive Bayes - ', nb(x, y))
print('XGB - ', xgbb(x, y))
print('ADB - ', adb(x, y))
print('MLP - ', mlp(x, y))
print('GB - ', gb(x, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  (0.9949066666666667, 0.9922268039664432)
dtc -  (1.0, 1.0)
rfc -  (1.0, 1.0)
knn -  (0.9999766666666666, 0.9999646546986016)
Naive Bayes -  (0.7935566666666667, 0.45216060613656067)
XGB -  (1.0, 1.0)
ADB -  (1.0, 1.0)
MLP -  (0.9999966666666666, 0.9999949506117756)
GB -  (1.0, 1.0)


In [31]:
def ica(x, i):
    ica = FastICA(n_components = i, random_state = 42)
    data_ica = ica.fit_transform(x)
    data_ica = pd.DataFrame(data_ica)
    return data_ica

In [32]:
def pca(x, i):
    pca = decomposition.PCA(n_components = i, random_state=42)
    data_pca = pca.fit_transform(x)
    data_pca = pd.DataFrame(data_pca)
    return data_pca

In [33]:
def mi(x, y):
    mi = sp(percentile = 50)
    data_mi = mi.fit(x, y)
    data_mi = mi.transform(x)
    data_mi = pd.DataFrame(data_mi)
    return data_mi

In [34]:
def smf(x, y):
    lasso_model = Lasso(alpha = 0.1)
    selector = SelectFromModel(lasso_model)
    X_selected = selector.fit_transform(x, y)
    selected_features_indices = selector.get_support(indices=True)
    selected_features_names = x.columns[selected_features_indices]
    return x[selected_features_names]

In [35]:
def sfs(x, y, model):
    sfs_selector = SequentialFeatureSelector(model, forward = True, k_features = len(x.columns) // 2)
    X_selected = sfs_selector.fit_transform(x, y)
    selected_features_names = list(x.columns[sfs_selector.k_feature_idx_])
    return x[selected_features_names]

In [36]:
def rfe(x, y, model):
    rfecv_selector = RFECV(estimator = model, step = 1, cv = 5, scoring = 'accuracy')
    X_selected = rfecv_selector.fit_transform(x, y)
    selected_features_names = list(x.columns[rfecv_selector.support_])
    return x[selected_features_names]

In [37]:
def cfs(x):
    corr_matrix = x.corr()
    high_corr_features = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > 0.8:
                colname = corr_matrix.columns[i]
                high_corr_features.append(colname)

    return x.drop(high_corr_features, axis=1)

In [38]:
def fir(x, y):
    rfc = RandomForestClassifier(n_estimators=100)
    rfc.fit(x, y)
    feature_importances = list(zip(x.columns, rfc.feature_importances_))
    a = sorted(feature_importances, key=lambda x: x[1], reverse=True)
    ls = []
    for i in a[:len(a)//2 + 1]: ls.append(i[0])
    return x[ls]

In [39]:
data_ica = ica(x, len(x.columns) // 2)

C:\Users\serev\anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


In [40]:
data_pca = pca(x, len(x.columns) // 2)

In [41]:
data_mi = mi(x, y)

C:\Users\serev\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [42]:
data_smf = smf(x, y)

In [43]:
data_cfs = cfs(x)

In [44]:
data_fir = fir(x, y)

In [45]:
#with feature selection ica


print('lr - ', lr(data_ica, y))
print('dtc - ', dtc(data_ica, y))
print('rfc - ', rfc(data_ica, y))
print('knn - ', knn(data_ica, y))
# print('svm - ', svm(data_ica, y))
print('Naive Bayes - ', nb(data_ica, y))
print('XGB - ', xgbb(data_ica, y))
print('ADB - ', adb(data_ica, y))
print('MLP - ', mlp(data_ica, y))
print('GB - ', gb(data_ica, y))

lr -  (0.7914866666666667, 0.44180438592900495)
dtc -  (0.99998, 0.9999697042057281)
rfc -  (0.9999933333333333, 0.9999899012830049)
knn -  (0.9999933333333333, 0.9999899012830049)
Naive Bayes -  (0.88267, 0.7766818068525343)
XGB -  (0.9999933333333333, 0.9999899012830049)
ADB -  (1.0, 1.0)
MLP -  (1.0, 1.0)
GB -  (0.9999466666666667, 0.9999192150198881)


In [46]:
#with feature selection pca


print('lr - ', lr(data_pca, y))
print('dtc - ', dtc(data_pca, y))
print('rfc - ', rfc(data_pca, y))
print('knn - ', knn(data_pca, y))
# print('svm - ', svm(data_pca, y))
print('Naive Bayes - ', nb(data_pca, y))
print('XGB - ', xgbb(data_pca, y))
print('ADB - ', adb(data_pca, y))
print('MLP - ', mlp(data_pca, y))
print('GB - ', gb(data_pca, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  (1.0, 1.0)
dtc -  (1.0, 1.0)
rfc -  (1.0, 1.0)
knn -  (0.9999833333333333, 0.9999747530588774)
Naive Bayes -  (0.7935566666666667, 0.45216060613656067)
XGB -  (1.0, 1.0)
ADB -  (1.0, 1.0)
MLP -  (0.9999966666666666, 0.9999949506117756)
GB -  (1.0, 1.0)


In [47]:
#with feature selection mi


print('lr - ', lr(data_mi, y))
print('dtc - ', dtc(data_mi, y))
print('rfc - ', rfc(data_mi, y))
print('knn - ', knn(data_mi, y))
# print('svm - ', svm(data_mi, y))
print('Naive Bayes - ', nb(data_mi, y))
print('XGB - ', xgbb(data_mi, y))
print('ADB - ', adb(data_mi, y))
print('MLP - ', mlp(data_mi, y))
print('GB - ', gb(data_mi, y))

lr -  (1.0, 1.0)
dtc -  (1.0, 1.0)
rfc -  (1.0, 1.0)
knn -  (0.9999933333333333, 0.9999899012830049)
Naive Bayes -  (1.0, 1.0)
XGB -  (1.0, 1.0)
ADB -  (1.0, 1.0)
MLP -  (1.0, 1.0)
GB -  (1.0, 1.0)


In [48]:
#with feature selection smf


print('lr - ', lr(data_smf, y))
print('dtc - ', dtc(data_smf, y))
print('rfc - ', rfc(data_smf, y))
print('knn - ', knn(data_smf, y))
# print('svm - ', svm(data_smf, y))
print('Naive Bayes - ', nb(data_smf, y))
print('XGB - ', xgbb(data_smf, y))
print('ADB - ', adb(data_smf, y))
print('MLP - ', mlp(data_smf, y))
print('GB - ', gb(data_smf, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  (1.0, 1.0)
dtc -  (1.0, 1.0)
rfc -  (1.0, 1.0)
knn -  (0.9999866666666667, 0.9999798023281921)
Naive Bayes -  (0.9999966666666666, 0.9999949506117756)
XGB -  (1.0, 1.0)
ADB -  (1.0, 1.0)
MLP -  (0.9999966666666666, 0.9999949506117756)
GB -  (1.0, 1.0)


In [49]:
#with feature selection csf


print('lr - ', lr(data_cfs, y))
print('dtc - ', dtc(data_cfs, y))
print('rfc - ', rfc(data_cfs, y))
print('knn - ', knn(data_cfs, y))
# print('svm - ', svm(data_cfs, y))
print('Naive Bayes - ', nb(data_cfs, y))
print('XGB - ', xgbb(data_cfs, y))
print('ADB - ', adb(data_cfs, y))
print('MLP - ', mlp(data_cfs, y))
print('GB - ', gb(data_cfs, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  (0.99396, 0.9907515017325622)
dtc -  (1.0, 1.0)
rfc -  (1.0, 1.0)
knn -  (0.99998, 0.9999697038490144)
Naive Bayes -  (0.7935566666666667, 0.45216060613656067)
XGB -  (1.0, 1.0)
ADB -  (1.0, 1.0)
MLP -  (0.9999966666666666, 0.9999949506117756)
GB -  (1.0, 1.0)


In [50]:
#with feature selection fir


print('lr - ', lr(data_fir, y))
print('dtc - ', dtc(data_fir, y))
print('rfc - ', rfc(data_fir, y))
print('knn - ', knn(data_fir, y))
# print('svm - ', svm(data_fir, y))
print('Naive Bayes - ', nb(data_fir, y))
print('XGB - ', xgbb(data_fir, y))
print('ADB - ', adb(data_fir, y))
print('MLP - ', mlp(data_fir, y))
print('GB - ', gb(data_fir, y))

lr -  (1.0, 1.0)
dtc -  (1.0, 1.0)
rfc -  (1.0, 1.0)
knn -  (1.0, 1.0)
Naive Bayes -  (1.0, 1.0)
XGB -  (1.0, 1.0)
ADB -  (1.0, 1.0)
MLP -  (1.0, 1.0)
GB -  (1.0, 1.0)
